# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Folding your way to success”

The collections `fold` function is as important as the `Aggregate` method in the world of LINQ. My huge [tabulation for lesson 15](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/15-working-with-collections-in-fsharp.ipynb) shows that `Aggregate` is the equivalent of `fold`:


In [ ]:
static double Sum(IEnumerable<double> inputs) => inputs.Aggregate(0d, (state, input) =>
{
    var newState = state + input;
    Console.WriteLine($"Current state is {state}, input is {input}, new state value is {newState}");
    return newState;
});

Sum(Enumerable.Range(1, 5).Select(i => Convert.ToDouble(i)))

Current state is 0, input is 1, new state value is 1
Current state is 1, input is 2, new state value is 3
Current state is 3, input is 3, new state value is 6
Current state is 6, input is 4, new state value is 10
Current state is 10, input is 5, new state value is 15


15

In [ ]:
#!fsharp

let sum inputs =
    Seq.fold
        (fun state input ->
            let newState = state + input
            printfn $"Current state is {state}, input is {input}, new state value is {newState}"
            newState
        ) // 1st argument: fold function (“folder”)
        0 // 2nd argument: initial state
        inputs // 3rd argument: input collection

sum [1 .. 5]

Current state is 0, input is 1, new state value is 1
Current state is 1, input is 2, new state value is 3
Current state is 3, input is 3, new state value is 6
Current state is 6, input is 4, new state value is 10
Current state is 10, input is 5, new state value is 15


15

We see that `Seq.fold` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#fold)] takes _three_ arguments just like how `Aggregate` takes three arguments when it accepts a initial _seed_ value. We have already learned about the pipe operator, `|>`, which would handle our `inputs` argument; however Isaac Abraham can do _one_ better with the _double pipeline operator_, `||>`:

>The double pipeline operator acts the same as the normal pipeline, but it takes in the last _two_ arguments and moves them tot he front _as a tuple_.

In [ ]:
#!fsharp

let sum inputs =
    (0, inputs) ||> Seq.fold
        (fun state input ->
            let newState = state + input
            printfn $"Current state is {state}, input is {input}, new state value is {newState}"
            newState
        )

sum [1 .. 5]

Current state is 0, input is 1, new state value is 1
Current state is 1, input is 2, new state value is 3
Current state is 3, input is 3, new state value is 6
Current state is 6, input is 4, new state value is 10
Current state is 10, input is 5, new state value is 15


15

## Using related fold functions

We have used `Seq.fold` to implement `Seq.sum` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#sum)] which strongly implies that `Seq.fold` is related to `Seq.sum` and other _aggregate_ functions, like `Seq.average` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#average)] and:

- `Seq.foldBack` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#foldBack)]
- `Seq.mapFold` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#mapFold)]
- `Seq.reduce` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#reduce)]
- `Seq.scan` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#scan)]
- `Seq.unfold` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#unfold)]

Let’s visit all of these:

In [ ]:
#!fsharp

[1 .. 5] |> Seq.sum

15

In [ ]:
#!fsharp

[ 2.0; 2.0; 2.0; 5.0 ] |> Seq.average

2.75

### `Seq.foldBack`

In [ ]:
#!fsharp

let sumBack inputs =
    (inputs, 0) ||> Seq.foldBack
        (fun state input ->
            let newState = state + input
            printfn $"Current state is {state}, input is {input}, new state value is {newState}"
            newState
        )

sumBack [1 .. 5]

Current state is 5, input is 0, new state value is 5
Current state is 4, input is 5, new state value is 9
Current state is 3, input is 9, new state value is 12
Current state is 2, input is 12, new state value is 14
Current state is 1, input is 14, new state value is 15


15

### `Seq.mapFold`

In [ ]:
#!fsharp

let runningSumAndTotal inputs =
    (0, inputs) ||> Seq.mapFold
        (fun state input ->
            let newState = state + input
            printfn $"Current state is {state}, input is {input}, new state value is {newState}"
            (newState, newState) // the first item of the tuple is the running total
        )

runningSumAndTotal [1 .. 5]

Current state is 0, input is 1, new state value is 1
Current state is 1, input is 2, new state value is 3
Current state is 3, input is 3, new state value is 6
Current state is 6, input is 4, new state value is 10
Current state is 10, input is 5, new state value is 15


Item1,Item2
"[ 1, 3, 6, 10, 15 ]",15


### `Seq.reduce`

In [ ]:
#!fsharp

let reduce inputs =
    inputs |> Seq.reduce
        (fun state input ->
            let newState = state + input
            printfn $"Current state is {state}, input is {input}, new state value is {newState}"
            newState
        )

reduce [1 .. 5]

Current state is 1, input is 2, new state value is 3
Current state is 3, input is 3, new state value is 6
Current state is 6, input is 4, new state value is 10
Current state is 10, input is 5, new state value is 15


15

We see that `reduce` is a simplified version of `fold` that uses the first element in the collection as initial state and can support the old `|>` operator. This ‘simplification’ is not without unexpected nuance as this [StackOverflow answer](https://stackoverflow.com/a/9055928/22944) describes:

>The fact that `fold` takes an explicit initial value for the accumulator also means that the result of the `fold` function can have a different type than the type of values in the list. …When using `reduce`, the type of accumulator is the same as the type of values in the list—this means that if you have a list of numbers, the result will have to be a number.

The only way to get around this limitation (or this _simplicity_) is to use `map` with `reduce` (see “Composing functions with `fold`” below).

### `Seq.scan`

In [ ]:
#!fsharp

let runningSum inputs =
    (0, inputs) ||> Seq.scan
        (fun state input ->
            let newState = state + input
            printfn $"Current state is {state}, input is {input}, new state value is {newState}"
            newState
        )

runningSum [1 .. 5]

Current state is 0, input is 1, new state value is 1
Current state is 1, input is 2, new state value is 3
Current state is 3, input is 3, new state value is 6
Current state is 6, input is 4, new state value is 10
Current state is 10, input is 5, new state value is 15


index,value
0,0
1,1
2,3
3,6
4,10
5,15


### `Seq.unfold`

In [ ]:
#!fsharp

let range upperLimit =
    0 |> Seq.unfold
        (fun state ->
            if state > upperLimit then None
            else Some(state, state + 1) // see https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/options
        )

range 5

index,value
0,0
1,1
2,2
3,3
4,4
5,5


## Folding instead of while loops

The “Folding instead of while loops” section of the book has at least two goals:

1. replacing imperative looping with `fold`
2. introducing sequence expressions

To make things easier for myself in these notes. I will separate these two goals and try to meet them here.

### replacing imperative looping with `fold`

The Blog of Reed Copsey, Jr. does a great job in “[F# Basics—From loops to folds](https://reedcopsey.com/2019/12/19/f-basics-from-loops-to-folds/),” showing how to replace imperative looping with `fold`. He leads us to this succinct-but-`mutable`-ridden `for` loop:

In [ ]:
#!fsharp

let addNewConditionally last number =
    if number % 3 = 0 then last + number else last

let mutable sum = 0
for number in { 1..20 } do
    sum <- addNewConditionally sum number

printfn "The sum is %d" sum

The sum is 63


We can use `fold` to remove the side effects of mutation:

In [ ]:
#!fsharp

let addNewConditionally last number =
    if number % 3 = 0 then last + number else last

let sum = { 1 .. 20 } |> Seq.fold addNewConditionally 0

printfn "The sum is %d" sum

The sum is 63


Remembering what Isaac Abraham taught us about `||>` we can make a slight change to the above:

In [ ]:
#!fsharp

let addNewConditionally last number =
    if number % 3 = 0 then last + number else last

let sum = (0, { 1 .. 20 }) ||> Seq.fold addNewConditionally

printfn "The sum is %d" sum

The sum is 63


### introducing sequence expressions

My notes for [lesson 13](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/13-achieving-code-reuse-in-fsharp.ipynb) introduced _computation expressions_ which is Microsoft’s current way of referring to what Isaac called around 2018 _sequence expressions_ (as well as _computation expressions_). The Microsoft docs has some interesting points, like we can omit `yield` and replace it with `->`—so can change this:

In [ ]:
#!fsharp

let squares =
    seq {
        for i in 1..10 do
            yield i * i
    }
squares |> printf "%A"

seq [1; 4; 9; 16; ...]

To this one-liner:

In [ ]:
#!fsharp

let squares = seq { for i in 1..10 -> i * i }
squares |> printf "%A"

seq [1; 4; 9; 16; ...]

## Composing functions with `fold`

The `fold` can be used when functions are treated like data and collected in, say, a `List`:

In [ ]:
#!fsharp

open System

type Rule = string -> bool * string // `Rule` is type alias

let rules : Rule list = // the type could also be written as `list<Rule>`
    [
        fun text -> ((text.Split ' ').Length = 3, "Must be three words")
        fun text -> (text.Length <= 30, "Max length is 30 characters")
        fun text -> (text |> Seq.filter Char.IsLetter |> Seq.forall Char.IsLetter, "All letters must be caps")
    ]

We can resort to looping to remind ourselves of how `fold` can be used here:

In [ ]:
#!fsharp

let test = "THIS, A TEST"

for rule in rules do
    rule test |> printf "%A\n"


(true, "Must be three words")
(true, "Max length is 30 characters")
(true, "All letters must be caps")


In [ ]:
#!fsharp

let myFolder (rule: Rule) text =
    let r, m = rule text
    if r then r else failwith m

let result =
    (true, rules) // we need `true` here for initial state 😐
    ||> List.fold (fun previousResult rule -> previousResult && myFolder rule test)

result

True

Composing functions with `fold` looks a bit awkward as written above. We have this silly and confusing use of `true`, working with the `||>` operator. We do not need an initial state for this rules engine. According to the book, and seen above we can use `List.reduce` to eliminate the initial state:

In [ ]:
#!fsharp

let result =
    rules
    |> List.map (fun rule -> fst (rule test)) // map `rules` to `bool` results
    |> List.reduce (fun previousResult result -> previousResult && result) // reduce the results to a single Boolean result

result

True

Alternatively, we can change the above to a function that can `validate` any `text`, composing our rules into a single function:

In [ ]:
#!fsharp

let validate text =
    rules
    |> List.map (fun rule -> fst (rule text))
    |> List.reduce (fun previousResult result -> previousResult && result)

validate test

True

This combination of `map` and `reduce` is said to be the inspiration behind [MapReduce](https://en.wikipedia.org/wiki/MapReduce) which is a specialization of the [_split-apply-combine strategy_](https://www.jstatsoft.org/article/view/v040i01) for data analysis. In that world of data science, the `rule` functions would run in parallel on different machines.

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.246201+da749355d416da20e634e5c80073b92356b57e0eBuild date: 2021-09-12T07:21:44.0000000Zhttps://github.com/dotnet/interactive
